In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [3]:
#data load
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

In [4]:
#Preprocessing the data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        print(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        print(documents)
        # add to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

['Hi', 'there']
[(['Hi', 'there'], 'greeting')]
['How', 'are', 'you']
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting')]
['Is', 'anyone', 'there', '?']
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting')]
['Hey']
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting')]
['Hola']
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting')]
['Hello']
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting')]
['Good', 'day']
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting')

In [5]:
# lemmatize
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))


In [6]:
print(words)

["'s", 'address', 'admission', 'and', 'anyone', 'are', 'available', 'awesome', 'be', 'bye', 'can', 'capitation', 'chatting', 'contact', 'could', 'day', 'detail', 'do', 'email', 'facility', 'fee', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'id', 'is', 'kg', 'later', 'location', 'me', 'next', 'nice', 'no', 'number', 'of', 'offered', 'phone', 'process', 'provide', 'regulation', 'rule', 'school', 'see', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'vision', 'what', 'you']


In [7]:
# sort classes
classes = sorted(list(set(classes)))

In [8]:
print(classes)

['address', 'admissionprocess', 'contact', 'facilities', 'fees', 'goodbye', 'greeting', 'options', 'ruleregulation', 'thanks', 'vision']


In [9]:
# documents-patterns and intents
print (len(documents), "documents")

49 documents


In [10]:
# classes-intents
print (len(classes), "classes", classes)

11 classes ['address', 'admissionprocess', 'contact', 'facilities', 'fees', 'goodbye', 'greeting', 'options', 'ruleregulation', 'thanks', 'vision']


In [11]:
print (len(words), "unique lemmatized words", words)

63 unique lemmatized words ["'s", 'address', 'admission', 'and', 'anyone', 'are', 'available', 'awesome', 'be', 'bye', 'can', 'capitation', 'chatting', 'contact', 'could', 'day', 'detail', 'do', 'email', 'facility', 'fee', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'id', 'is', 'kg', 'later', 'location', 'me', 'next', 'nice', 'no', 'number', 'of', 'offered', 'phone', 'process', 'provide', 'regulation', 'rule', 'school', 'see', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'vision', 'what', 'you']


In [12]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [13]:
# creating training data
training = []

In [14]:
#empty array for output
output_empty = [0] * len(classes)

In [15]:
# training set
for doc in documents:
    # initialize 
    bag = []
    # list of tokenized words for pattern
    pattern_words = doc[0]
    print(pattern_words)
    # lemmatize words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
    # output is'0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    print(output_row)
    training.append([bag, output_row])
    

['Hi', 'there']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
['How', 'are', 'you']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
['Is', 'anyone', 'there', '?']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
['Hey']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
['Hola']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
['Hello']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
['Good', 'day']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
['Bye']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['See', 'you', 'later']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['Goodbye']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['Nice', 'chatting', 'to', 'you', ',', 'bye']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['Till', 'next', 'time']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['Thanks']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
['Thank', 'you']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
['That', "'s", 'helpful']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
['Awesome', ',', 'thanks']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
['Thanks', 'for', 'helping', 'me']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
['How', 'you', 'could', 'help', 'me', '?']
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
['

In [16]:
print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]], [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [17]:
# shuffle,turn into np.array
random.shuffle(training)
training = np.array(training)

C:\Users\ktman\AppData\Local\Temp\ipykernel_2056\2194830951.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [18]:
# train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print(train_x)
print(train_y)
print("Training data created")

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [42]:
# model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [43]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
#SGd is a learning and optimization algorithm that trains algorithms in ML- machine learning
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [44]:
#fitting and saving the model 
#An epoch in machine learning means one complete pass of the training dataset through the algorithm.
#save the model with name chatbot_model.h5
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)


Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.3638 - accuracy: 0.1224
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 2.3944 - accuracy: 0.1633
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2925 - accuracy: 0.1633
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2613 - accuracy: 0.2449
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1893 - accuracy: 0.3061
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0566 - accuracy: 0.3469
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9881 - accuracy: 0.3673
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9654 - accuracy: 0.3878
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7854 - accuracy: 0.5102
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8333 - accuracy: 0.5306
Epoch 11/

In [22]:
print("model created")

model created
